# Transfer learning
* Download data
* Get a pretrained model
* Do training
* Load results from tensorboard writer

In [1]:
import torch
import torchvision
from torch import nn
from torchvision import transforms

print(f"torch {torch.__version__} | torchvision {torchvision.__version__}")

torch 2.0.1+cu118 | torchvision 0.15.2+cu118


In [2]:
# import torchinfo
try:
    from torchinfo import summary
except:
    print(f"[INFO] installing torchinfo...")
    !pip install torchinfo
    from torchinfo import summary

In [3]:
# import modules from sources
from sources import utils, datasetup, engine, models

2023-11-06 20:23:48.145091: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-06 20:23:48.185937: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-06 20:23:48.809678: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
# device agnostic
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [5]:
# download data
source = "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip"
image_path = datasetup.download_data(source=source)
image_path

[INFO] data/images exist.


PosixPath('data/images')

In [6]:
# setup train and test directories
train_dir = image_path / "train"
test_dir = image_path / "test"
train_dir, test_dir

(PosixPath('data/images/train'), PosixPath('data/images/test'))

In [7]:
# setup an efficient net pretrained model
model, model_transforms = models.create_effnet(
    effnet_version=0
)
model_transforms

[INFO] creating EfficientNet_B0...


ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [8]:
# create dataloaders
train_dataloader, test_dataloader, class_names = datasetup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    train_transforms=model_transforms,
    test_transforms=model_transforms,
    batch_size=32
)

[INFO] creating dataloaders... 
train_dataloader: <torch.utils.data.dataloader.DataLoader object at 0x7fc4993f9810> 
test_dataloader: <torch.utils.data.dataloader.DataLoader object at 0x7fc4993f9b10> 
number of class_names: 3


In [9]:
# setup loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-3)

In [10]:
import shutil
# # shutil.rmtree("runs")
shutil.rmtree("writerLog")

In [11]:
# train pretrained model with new data
results = engine.train_tsb_writer(
    model=model,
    train_dataloader = train_dataloader,
    test_dataloader = test_dataloader,
    loss_fn = loss_fn,
    optimizer = optimizer,
    epochs=20,
    device=device,
    writer=utils.create_writer()
)

[INFO] creating writer log at: writerLog


  0%|          | 0/20 [00:00<?, ?it/s]

epoch: 0 | train_loss: 1.0589 | train_acc: 0.3750 | test_loss: 0.8518 | test_acc: 0.7633
epoch: 1 | train_loss: 0.9001 | train_acc: 0.5938 | test_loss: 0.8035 | test_acc: 0.7121
epoch: 2 | train_loss: 0.7379 | train_acc: 0.7109 | test_loss: 0.6512 | test_acc: 0.8655
epoch: 3 | train_loss: 0.6696 | train_acc: 0.7422 | test_loss: 0.6386 | test_acc: 0.8968
epoch: 4 | train_loss: 0.6478 | train_acc: 0.7773 | test_loss: 0.5551 | test_acc: 0.9167
epoch: 5 | train_loss: 0.5816 | train_acc: 0.8008 | test_loss: 0.5459 | test_acc: 0.8561
epoch: 6 | train_loss: 0.5242 | train_acc: 0.8047 | test_loss: 0.5377 | test_acc: 0.8570
epoch: 7 | train_loss: 0.5605 | train_acc: 0.7969 | test_loss: 0.5494 | test_acc: 0.8456
epoch: 8 | train_loss: 0.4769 | train_acc: 0.8320 | test_loss: 0.5043 | test_acc: 0.8456
epoch: 9 | train_loss: 0.4124 | train_acc: 0.9531 | test_loss: 0.4374 | test_acc: 0.8769
epoch: 10 | train_loss: 0.4823 | train_acc: 0.8281 | test_loss: 0.4325 | test_acc: 0.8968
epoch: 11 | train_lo

In [ ]:
%load_ext tensorboard
%tensorboard --logdir writerLog/ --port 0